# 0. Initialize

## 0.1. Import Libraries

In [65]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys, glob
import gzip
import random
import tqdm
import json
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
from datetime import datetime, timezone

from IPython import display
import matplotlib as mpl
from matplotlib import pyplot as plt

from IPython.display import clear_output

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


resources used:

    https://medium.com/@jad.elderfield/feature-generation-from-tweets-9af0565ad6e6

## 0.2. DEFINE VARIABLES 

In [13]:
DATA_PATH = 'D:\Qsync\BILGEHAN\sabanci\CS\CS412\project\part2\\' # '<insert-your-training-data-path-here>'

ROUND = 1 # This project will have 3 rounds of predictions: 1,2,3
STUDENT_ID = '28452'#'<insert-your-id-here>'
PROJECT_CODE = 'CS412de753a916abf'#'<insert-your-code-here>' # Same code for the annotation eg. CS412xxxxx

## 0.3. Read Training & Evaluation Data

### 0.3.1. Get the labels for tweets

In [113]:
#trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH))
trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH), dtype={'tweet_id': str, 'isPolitical': str})
trainingTweetDf

,tweet_id,isPolitical
0,1597170281545551872,Yes
1,1431700027471192069,No
2,1566035577090281472,Yes
3,1591538690869940225,Yes
4,1583898169238167554,Yes
...,...,...
3795,1596978345560150016,Yes
3796,1582399680343789568,Yes
3797,1420407025847132171,No
3798,1589993032975544320,Yes


In [110]:
trainingTweetDf.isPolitical.value_counts()

Yes    2442
No     1353
Name: isPolitical, dtype: int64

### 0.3.2. Get the labels for users

In [16]:
trainingUserDf = pd.read_csv('{}training-user.csv'.format(DATA_PATH))
#trainingUserDf = pd.read_csv('training-user.csv')
trainingUserDf

,screen_name,isBot
0,koftecancaddy,No
1,ahaber,No
2,selahat03949652,No
3,erdin06357062,No
4,bhct__necatii,No
...,...,...
3795,ddionysos_s,No
3796,alpar_kaan,No
3797,nedenburdaysam,No
3798,muhibban_eco,No


In [17]:
trainingUserDf.isBot.value_counts()

No          3027
Yes          690
Not sure      78
Name: isBot, dtype: int64

### 0.3.3. Expand your dataset with metadata and tweets

In [18]:
# You can also expand training data by downloading your own labeled datasets following the link
# Download the documents under "Link to training data"

print('http://www.onurvarol.com/Annotation-CS412-202201/reports/report_{}.html'.format(PROJECT_CODE))

http://www.onurvarol.com/Annotation-CS412-202201/reports/report_CS412de753a916abf.html


# 1. EXTRACT FEATURES
Under *1.1. Political Tweet Detection* and *1.2. Bot Detection*, we firstly collect raw data for processing. We then combine some of them (total_interactions = num_favorites + num_retweets) or use them to extract features (whether the tweet has one of the political entities @meralaksener, @kilicdarogluk etc.).

We expect you to collect more raw data from **tweet_metadata**, **user_profiles** and **user_tweets** files by creating a function as shown in below examples such as *check_if_retweet()* and using it while iterating over data as shown under *Merge Collected Features*.

We also expect you to create new variables as much as you can from the data in order to make your predictions more accurate. For example, you may want to check:

- The tweet sources that a user frequently uses
- Whether the user is a verified account or not

...

to assess whether **a user is a bot or not** and whether **a tweet is political or not**.

In [19]:
PATH_TO_DOWNLOADED = DATA_PATH # 'D:\\Qsync\\BILGEHAN\\sabanci\\CS\\CS412\\project\\part2\\'

## 1.1. Political Tweet Detection
This part stands for the feature extraction of tweets. We start with collecting the raw data from *tweet_metadata*, then use some of them to extract features.

### 1.1.1. Get Raw Data

#### 1.1.1.1. Check if Retweet

In [21]:
def check_if_retweet(tweet_metadata_line):
    is_retweet = 0
    retweeted_username = None

    try:
        tweet_metadata_line['retweeted_status'] #check if "retweeted_status" exists (may or may not, no guarantee). If so, return info on it.
        retweeted_username = tweet_metadata_line['retweeted_status']['user']['screen_name'].lower()
        is_retweet = 1

    except KeyError:
        pass

    return is_retweet, retweeted_username

#### 1.1.1.2. Get Tweet Text

In [22]:
def get_tweet_text(tweet_metadata_line):
    text = tweet_metadata_line['text']
    
    return text

#### 1.1.1.3. Get Tweet ID

In [23]:
def get_tweet_id(tweet_metadata_line):
    id_str = tweet_metadata_line['id_str']
    
    return id_str

#### 1.1.1.4. Get Number of Mentions and Hashtags

In [24]:
def get_number_mentions_hashtags(tweet_metadata_line):
    num_mentions = len(tweet_metadata_line['entities']['user_mentions'])
    num_hashtags = len(tweet_metadata_line['entities']['hashtags'])

    return num_mentions, num_hashtags

#### 1.1.1.5. Get Number of Retweets and Favorites

In [25]:
def get_number_retweets_favorites(tweet_metadata_line):
    retweet_count = tweet_metadata_line['retweet_count']
    favorite_count = tweet_metadata_line['favorite_count']
    
    return retweet_count, favorite_count

#### 1.1.1.6. Get User Info

In [26]:
def get_user_info(tweet_metadata_line):
    id = tweet_metadata_line['user']['id_str']
    screen_name = tweet_metadata_line['user']['screen_name'].lower()
    description = tweet_metadata_line['user']['description']

    return id, screen_name, description

#### 1.1.1.7. Check if tweet was made as a reply to another user

In [27]:
def check_if_reply(tweet_metadata_line):
    is_reply = 0
    parent_username = None

    try:
        parent_username = tweet_metadata_line['in_reply_to_screen_name']
        if parent_username:
            parent_username = parent_username.lower()
            is_reply = 1

    except KeyError:
        pass

    return is_reply, parent_username

#### 1.1.1.8. Get user location at the time of tweet

In [49]:
def get_user_location(tweet_metadata_line): #no guarantee of this being simple. ex. "Meram, Konya", "Ankara"
    location = tweet_metadata_line['user']['location']

#    if(location == ""):#no location info
#        return False

    return location.lower()

#### 1.1.1.9. Is the user verified?

In [29]:
def get_verified(tweet_metadata_line):
    verified = tweet_metadata_line['user']['verified']
    return verified

#### 1.1.1.10. Age of account

In [30]:
def get_age(tweet_metadata_line):
    str_age = tweet_metadata_line['user']['created_at'] # ex. "Tue Sep 06 13:22:17 +0000 2020",
    age = datetime.strptime(str_age, '%a %b %d %H:%M:%S %z %Y') #date formats: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes

    return age

#### 1.1.1.11. Get friends and followers

In [31]:
def get_friends_followers(tweet_metadata_line):
    friends = tweet_metadata_line['user']['friends_count']
    followers = tweet_metadata_line['user']['followers_count']

    return friends, followers

#### 1.1.1.12 Number of tweets they have made

In [32]:
def get_tweet_count(tweet_metadata_line): #can be used along with age to see tweets per day ratio (how active they are)
    tweets = tweet_metadata_line['user']['statuses_count']
    return tweets

#### 1.1.1.13 Get how many times they have been listed (tells you how influential the account is)

In [33]:
def get_listed_count(tweet_metadata_line):
    listedCount = tweet_metadata_line['user']['listed_count']
    return listedCount

### 1.1.2. Derive Manually Crafted Features

#### 1.1.2.1. Check for political entity in text

In [34]:
def check_political_ent(text):
    
    # the list below can be modified and some new names may be added (or removed)
    list_of_entities = ['meral_aksener', 'kilicdarogluk', 'vekilince', 'RTErdogan', 'MevlutCavusoglu', 'umitozdag',
                        'eczozgurozel', 'ekrem_imamoglu', 'mansuryavas06', 'suleymansoylu', 'T_Karamollaoglu', 'DurmusYillmaz',
                        'erkbas', 'TCAytunCiray', 'tuncsoyer', 'dbdevletbahceli', 'bybekirbozdag', 'NureddinNebati', 'mkulunk',
                        'celebimehmeta', 'drfahrettinkoca', 'alibabacan', 'ahmet_davutoglu', 'erbakanfatih']
    
    entities_in_text = [ent for ent in list_of_entities if ent.lower() in text.lower()]
    number_entities = len(entities_in_text)

    return number_entities

#### 1.1.2.2. Number of total interactions

In [35]:
def total_interactions(retweet_count, favorite_count):
    total_num_interactions = retweet_count + favorite_count
    
    return total_num_interactions

#### 1.1.2.3. Amount of numbers in username

In [36]:
def UsernameNumberCount(displayName):
    numberCount = 0
    
    for i in displayName:
        if i.isnumeric():
            numberCount += 1

    return numberCount

In [37]:
def UsernameSpecialCharacterCount(displayName):
    specialCount = 0
    
    for i in displayName:
        if not i.isalpha():
            specialCount += 1

    return specialCount

#### 1.1.2.4. Friends to Followers ratio

In [46]:
def FriendsVFollowers(friends, followers):
    return (friends+1)/(followers+1)

#### 1.1.2.5. Account activeness

In [42]:
def TweetsPerDay(tweetCount, age):
    now = datetime.now(timezone.utc)
    deltaTime = now-age

    return tweetCount/deltaTime.days

#can this be improved? ratio will be smaller if this is run further in the future!
#This is unacceptable, is there a better way?

### 1.1.2. Collect data using the functions above and transform into a Pandas DataFrame

In [55]:
dfPolitical = {'tweet_id':[],
              'is_retweet':[],
              'retweeted_username':[],
              'text':[],
              'num_mentions':[],
              'num_hashtags':[],
              'num_retweets':[],
              'num_favorites':[],
              'user_id':[],
              'user_screen_name':[],
              'user_description':[],
              'num_political_entities':[],
              'total_interactions':[],
              'total_special_char_in_username':[],
              'friend_ratio':[],
              'account_activeness':[],
              'is_reply':[],
              'location':[],
              'is_verified':[],
              'account_age':[],
              'friend_count':[],
              'follower_count':[],
              'tweets_count':[],
              'tweets_listed_count':[]}


with gzip.open(f"{PATH_TO_DOWNLOADED}tweet_metadata.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)
        
        # raw data:
        id_str = get_tweet_id(line)
        is_retweet, retweeted_username = check_if_retweet(line)
        text = get_tweet_text(line)
        num_mentions, num_hashtags = get_number_mentions_hashtags(line)
        retweet_count, favorite_count = get_number_retweets_favorites(line)
        user_id_str, screen_name, user_description = get_user_info(line)
        is_reply = check_if_reply(line)#not used or placed in df
        location = get_user_location(line)#not used or placed in df
        is_verified = get_verified(line)#not used or placed in df
        account_age = get_age(line)#used but not placed in df
        friend_count, follower_count = get_friends_followers(line)#used but not placed in df
        tweets_count = get_tweet_count(line)#used but not placed in df
        tweets_listed_count = get_listed_count(line)#not used or placed in df

        # manually crafted data:
        num_political_entities = check_political_ent(text)
        total_num_interactions = total_interactions(retweet_count, favorite_count)
        total_special_char_in_username = UsernameSpecialCharacterCount(screen_name)
        friend_ratio = FriendsVFollowers(friend_count, follower_count)
        account_activeness = TweetsPerDay(tweets_count, account_age)

        dfPolitical['tweet_id'].append(id_str)
        dfPolitical['is_retweet'].append(is_retweet)
        dfPolitical['retweeted_username'].append(retweeted_username)
        dfPolitical['text'].append(text)
        dfPolitical['num_mentions'].append(num_mentions)
        dfPolitical['num_hashtags'].append(num_hashtags)
        dfPolitical['num_retweets'].append(retweet_count)
        dfPolitical['num_favorites'].append(favorite_count)
        dfPolitical['user_id'].append(user_id_str)
        dfPolitical['user_screen_name'].append(screen_name)
        dfPolitical['user_description'].append(user_description)
        dfPolitical['num_political_entities'].append(num_political_entities)
        dfPolitical['total_interactions'].append(total_num_interactions)
        dfPolitical['total_special_char_in_username'].append(total_special_char_in_username)
        dfPolitical['friend_ratio'].append(friend_ratio)
        dfPolitical['account_activeness'].append(account_activeness)
        dfPolitical['is_reply'].append(is_reply)
        dfPolitical['location'].append(location)
        dfPolitical['is_verified'].append(is_verified)
        dfPolitical['account_age'].append(account_age)
        dfPolitical['friend_count'].append(friend_count)
        dfPolitical['follower_count'].append(follower_count)
        dfPolitical['tweets_count'].append(tweets_count)
        dfPolitical['tweets_listed_count'].append(tweets_listed_count)

In [56]:
dfPolitical = pd.DataFrame(dfPolitical)
dfPolitical

,tweet_id,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,total_interactions,total_special_char_in_username,friend_ratio,account_activeness,is_reply,location,is_verified,account_age,friend_count,follower_count,tweets_count,tweets_listed_count
0,1588568792984346624,0,None,"Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...",0,0,49,98,920963718103650304,maviruh_,shu/\nburaya afilli bir söz yazdığımı varsayın,0,147,1,1.894737,1.382600,"(0, None)",ankara,False,2017-10-19 10:43:55+00:00,539,284,2638,0
1,1588452263047069697,0,None,"@mahirunal Gavur İzmir ya onlar, hani Cumhuriy...",1,0,0,0,595514060,mtfdan,,0,0,0,4.469697,1.715355,"(1, mahirunal)",,False,2012-05-31 14:08:26+00:00,589,131,6647,3
2,1569589330544398336,0,None,#ŞehitAdayıUzmÇvşaKadro\nSiz İstesenizde Istem...,0,1,0,0,1356375754561490947,ahsucilginuzman,Vatan Sevdalisi,0,0,0,1.950820,4.141643,"(0, None)",,False,2021-02-01 22:56:06+00:00,118,60,2924,0
3,1570428119609139201,0,None,@ajans_muhbir Siz kaypak olmayıp onay vermesey...,1,0,0,0,1478775431008595968,hamitelkelle,HighOne,0,0,0,23.333333,4.831978,"(1, ajans_muhbir)",,False,2022-01-05 17:08:49+00:00,69,2,1783,0
4,1551163840368414722,0,None,Engelli öğretmenler olarak önümüzdeki engeller...,0,0,0,0,1511976696337113088,sed58417690,,0,0,8,1.391667,27.288809,"(0, None)",,False,2022-04-07 07:58:42+00:00,166,119,7559,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33528,1568595408233832448,0,None,Gerçek kimlik taşımayan hesaplara cevap vermem...,0,0,9,81,576247173,ardanzenturk,RT ONAYLADIĞIM ANLAMINA GELMEZ\nArtık fikirler...,0,90,0,0.024201,10.611396,"(0, None)",,False,2012-05-10 11:47:23+00:00,4147,171399,41342,379
33529,1584027427696959488,0,None,@umitozdag Neden Suriyelilerle ilgili bu kadar...,1,0,1,8,162308585,ozgul_61,Bridge design engineer Yaay hesabı : dilfiruz,1,9,3,0.819914,11.445441,"(1, umitozdag)",payi̇taht 🇹🇷,False,2010-07-03 07:58:43+00:00,3227,3936,52340,4
33530,1585945783307730945,0,None,@celebimehmeta Niye Türkiye yüzyılıda.Türkiye ...,1,0,0,1,415025519,ladrekova,,1,1,0,3.385246,0.300983,"(1, celebimehmeta)",,False,2011-11-17 20:15:55+00:00,412,121,1225,0
33531,1569748909521801221,1,muazzezeralp,RT @muazzezeralp: @Doan58213655 @denizkonur @N...,7,1,6,0,1442125177727307781,yapikytgrivrlsn,,1,6,0,1.695946,327.274468,"(0, None)",,False,2021-09-26 13:53:55+00:00,1003,591,153819,0


## 1.2. From Users

### 1.2.1. Get user metadata from user_profiles.jsons.gz

#### 1.2.1.1. Get user info metadata

In [57]:
def get_user_info_metadata(user_metadata_line):
    
    user_id = user_metadata_line['id_str']
    user_name = user_metadata_line['name']
    user_screen_name = user_metadata_line['screen_name'].lower()
    user_location = user_metadata_line['location']
    user_description = user_metadata_line['description']
    user_followers_count = user_metadata_line['followers_count']
    user_friends_count = user_metadata_line['friends_count']
    
    dictionary = {'user_id':user_id, 'user_name': user_name, 'user_screen_name':user_screen_name, 'user_location':user_location,
     'user_description':user_description, 'user_followers_count':user_followers_count, 'user_friends_count':user_friends_count}

    return dictionary

#### 1.2.1.2. Get followers/(followers+friends) ratio

In [58]:
def get_followers_all_ratio(user_followers_count, user_friends_count):
    
    if user_friends_count + user_followers_count == 0:
        followers_all_ratio = 0

    else:
        followers_all_ratio =  user_followers_count / (user_friends_count + user_followers_count)

    return followers_all_ratio

#### 1.2.1.3. Get description length

In [59]:
def get_desc_len(user_description):
    
    description_len = len(user_description)

    return description_len

In [60]:
dfBot = {'user_id':[],
         'user_name':[],
         'user_screen_name':[],
         'user_location':[],
         'user_description':[],
         'user_followers_count':[],
         'user_friends_count':[],
         'description_len':[],
         'followers_to_all_ratio':[]}

with gzip.open(f"{PATH_TO_DOWNLOADED}user_profiles.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)

        dictionary = get_user_info_metadata(line)
        for k,v in dictionary.items():
            dfBot[k].append(v)

        
        # manually crafted data:
        description_len = get_desc_len(dictionary['user_description'])
        dfBot['description_len'].append(description_len)
        
        followers_all_ratio = get_followers_all_ratio(dictionary['user_followers_count'], 
                                                      dictionary['user_friends_count'])

        dfBot['followers_to_all_ratio'].append(followers_all_ratio)

In [61]:
dfBot = pd.DataFrame(dfBot)
dfBot

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,0.732260
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,64,0.192308
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,65,0.325203
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,23,0.505051
...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,0,0.513453
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,100,0.975088
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,28,0.451362
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,116,0.314431


### 1.2.2. Get Tweet Info of Users in user_profiles.jsons.gz

#### 1.2.2.1. Check ratio of retweets to all tweets

In [62]:
def get_retweet_tweet_ratio(line):
    number_retweets = 0
    number_original_tweets = 0

    for tweet in line['tweets']:
        try:
            tweet['retweeted_status']
            number_retweets += 1
                
        except:
            number_original_tweets += 1
            
    total_tweets = number_retweets + number_original_tweets
    
    if total_tweets == 0:
        retweet_total_ratio = None
    else:
        retweet_total_ratio = number_retweets/(total_tweets)
    
    return retweet_total_ratio

#### 1.2.2.2. Check median number of favorites

In [63]:
def get_median_number_favorites(line):
    num_median_favorites = np.median([tweet['favorite_count'] for tweet in line['tweets']])

    return num_median_favorites

### 1.2.3. Collect data using the functions above and transform into a Pandas DataFrame

In [88]:
dfBotTweets = {'user_id':[],
               'retweet_total_ratio':[],
               'num_median_favorites':[],
               'num_of_tweets':[]
              }

i = 0

with gzip.open(f"{PATH_TO_DOWNLOADED}user_tweets.jsons.gz", "rb") as f:
    for line in f:

        line = json.loads(line)

        user_id = line['user_id']
        dfBotTweets['user_id'].append(user_id)
        
        retweet_total_ratio = get_retweet_tweet_ratio(line)
        dfBotTweets['retweet_total_ratio'].append(retweet_total_ratio)
        
        num_median_favorites = get_median_number_favorites(line)
        dfBotTweets['num_median_favorites'].append(num_median_favorites)
        
        dfBotTweets['num_of_tweets'].append(len(line['tweets']))

        i += 1
        if i % 100 == 0:
            clear_output(wait=True)
            print(i/283, "%. Processed", i , "lines")

101.07142857142857 %. Processed 28300 lines


In [89]:
dfBotTweets = pd.DataFrame(dfBotTweets)
dfBotTweets

,user_id,retweet_total_ratio,num_median_favorites,num_of_tweets
0,594642154,0.115000,2.0,200
1,525600289,0.005025,1.0,199
2,931895965501534209,0.900000,0.0,200
3,1591543462746329088,0.185000,0.0,200
4,734801354749796352,1.000000,0.0,200
...,...,...,...,...
28310,1591370361488252928,0.800000,0.0,200
28311,1475272459616235525,0.825000,0.0,200
28312,1096753792731750401,0.051020,1.0,196
28313,1269527617687953409,0.095000,2.0,200


### 1.2.3. Merge dfBot and dfBotTweets

In [90]:
dfBotAll = dfBot.merge(dfBotTweets,
                       how='left')

dfBotAll[['retweet_total_ratio', 'num_median_favorites']] = dfBotAll[['retweet_total_ratio', 'num_median_favorites']].fillna(0)

dfBotAll

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,retweet_total_ratio,num_median_favorites,num_of_tweets
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,0.395939,0.0,197.0
1,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,0.732260,0.125000,0.0,200.0
2,1116042038577958914,Yusuf Aksoy,yusufak63712920,,"Bir şeyden pişmanlık duymak istemiyorsan,her ş...",95,399,64,0.192308,0.910000,0.0,200.0
3,4859899931,Be (VIXX6) ama oppalarının düğününe gidemiyor,nedenburdaysam,Hufflepuff ortak salon,"SMStan\n/St☆rlight ///come on girls,this is ou...",40,83,65,0.325203,0.015306,1.0,196.0
4,2225373636,SLMDMR,biologselim,,BİYOLOG🔬🦠\nNanoteknoloji,100,98,23,0.505051,0.659898,0.0,197.0
...,...,...,...,...,...,...,...,...,...,...,...,...
29665,1320834618220781569,islammm,islam_mrsj,,,229,217,0,0.513453,0.015000,1.0,200.0
29666,111074128,Melda Onur,meldaonur,,"Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...",212457,5428,100,0.975088,0.291457,2.0,199.0
29667,36946875,ali ydm,ali_ydm,"İstanbul, Türkiye",hayat oyunsa bende jeton çok,116,141,28,0.451362,0.061538,0.0,195.0
29668,2389587396,Türkan Usta,turkanusta,"Ankara, Türkiye",Ustaya sormuşlar; hayatta yaptığın en büyük is...,1669,3639,116,0.314431,0.995000,0.0,200.0


# 2. TRAIN MODEL

## 2.1. Political Tweet Prediction

In [114]:
trainingTweetDf['tweet_id']

0       1597170281545551872
1       1431700027471192069
2       1566035577090281472
3       1591538690869940225
4       1583898169238167554
               ...         
3795    1596978345560150016
3796    1582399680343789568
3797    1420407025847132171
3798    1589993032975544320
3799    1595521343587508236
Name: tweet_id, Length: 3800, dtype: object

In [108]:
dfPolitical['tweet_id']

0        1588568792984346624
1        1588452263047069697
2        1569589330544398336
3        1570428119609139201
4        1551163840368414722
                ...         
33528    1568595408233832448
33529    1584027427696959488
33530    1585945783307730945
33531    1569748909521801221
33532    1585926877675282432
Name: tweet_id, Length: 33533, dtype: object

### 2.1.1. Merge dfPolitical data with labels

In [116]:
dfPoliticalAll_train = pd.merge(trainingTweetDf, dfPolitical, on='tweet_id')

dfPoliticalAll_train.head()

,tweet_id,isPolitical,is_retweet,retweeted_username,text,num_mentions,num_hashtags,num_retweets,num_favorites,user_id,user_screen_name,user_description,num_political_entities,total_interactions,total_special_char_in_username,friend_ratio,account_activeness,is_reply,location,is_verified,account_age,friend_count,follower_count,tweets_count,tweets_listed_count
0,1597170281545551872,Yes,0,None,@ankara_kusu Yeter ya ne Kılıçdaroğlu İmamoğlu...,1,0,0,0,1564335223395033089,cananko84528005,,0,0,8,0.701299,10.431818,"(1, ankara_kusu)",,False,2022-08-29 19:33:04+00:00,107,153,1377,0
1,1431700027471192069,No,0,None,@btwhocarees 10 dakikadır bunu düşünüyorum,0,0,0,0,1039600330521145350,selinanyjuice,,0,0,0,0.746130,4.501899,"(1, hithesackk)",,False,2018-09-11 19:43:28+00:00,240,322,7113,1
2,1566035577090281472,Yes,0,None,@RTErdogan #ÖncelikBizim\nİnşallah yıılarca em...,1,1,1,1,1340017324531789827,elif48806768,#Hak ve adalet elbet birgün yerini bulacak!. g...,1,2,8,0.632933,180.547270,"(1, rterdogan)",,False,2020-12-18 19:34:11+00:00,368,582,135591,0
3,1591538690869940225,Yes,0,None,SÜLEYMAN SOYLU ADAMDIR,0,0,0,2,501976568,_er_bayrak_,@_Er_Vatan Yedek Hesap\n ...,0,2,3,0.927768,3.481873,"(0, None)",türkiye,False,2012-02-24 17:37:44+00:00,2144,2311,13830,2
4,1583898169238167554,Yes,0,None,@Akparti Allah seni başımızdan eksik etmesin i...,1,0,0,1,1456959280297521156,ahin_yuksel,Araştır daha araştır ve sorgula. Zihin süzgeci...,0,1,1,1.304654,48.568765,"(1, akparti)",,False,2021-11-06 12:19:08+00:00,924,708,20836,0


### 2.1.2. Separate X and y values
We only use 3 features here to create a baseline model. However, it is not enough to get good results.

In [117]:
X = dfPoliticalAll_train[['num_political_entities','total_interactions','num_hashtags']]
y = dfPoliticalAll_train['isPolitical'].apply(lambda x: 1 if x=='Yes' else 0)

### 2.1.3. Train - validation split

In [118]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

### 2.1.4. Train the model

Here, you may use different models such as neural networks, XGBoost, AdaBoost, RandomForest, Linear Regression, Logistic Regression etc. to see which model does the best. Also, you can use grid_search_cv() or a basic for loop to optimize the hyperparameters of your model.

In [119]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance
dtc_political = DecisionTreeClassifier()

# fit your model
dtc_political.fit(X_train, y_train)

# make predictions
preds = dtc_political.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mse = mean_squared_error(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.35 
 Accuracy Score: 0.65 
 Confusion Matrix: 
 [[ 76 202]
 [ 64 418]]


## 2.2. Bot Detection

### 2.2.1. Merge dfBotAll data with labels

In [120]:
dfBotAll.user_screen_name = dfBotAll.user_screen_name.str.lower()

In [121]:
dfBotAll_train = dfBotAll.merge(trainingUserDf,
                               left_on='user_screen_name',
                               right_on='screen_name')

dfBotAll_train

,user_id,user_name,user_screen_name,user_location,user_description,user_followers_count,user_friends_count,description_len,followers_to_all_ratio,retweet_total_ratio,num_median_favorites,num_of_tweets,screen_name,isBot
0,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,0.395939,0.0,197.0,nasreenakhan006,Yes
1,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,0.395939,0.0,197.0,nasreenakhan006,Yes
2,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,0.395939,0.0,197.0,nasreenakhan006,No
3,1431241870848450577,Nasreena Khan Wazir,nasreenakhan006,"Islamabad, Pakistan",Student,65,185,7,0.260000,0.395939,0.0,197.0,nasreenakhan006,No
4,1304340303080386560,fania :((((,scorpiehoez,bogor,have a holly jolly🎄,8235,3011,19,0.732260,0.125000,0.0,200.0,scorpiehoez,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,1286770207134973954,Hamide Arabacı,anka6054,,,151,61,0,0.712264,0.000000,1.0,200.0,anka6054,No
3796,1598032338323214338,atamabekleyenbahceci,atamabekleyenzz,,,173,367,0,0.320370,0.580000,0.0,200.0,atamabekleyenzz,No
3797,760235343966863360,Emrah İNCİ,memrahinci,Istanbul - Bayburt,Researcher | Middle East | Political Science |...,5863,5905,71,0.498215,0.040000,36.0,200.0,memrahinci,No
3798,1553973684100124672,Murat Kkk,muratkkk18,,Normal sıradan bir insanım,1,10,26,0.090909,0.769231,0.0,13.0,muratkkk18,No


In [122]:
trainingUserDf.isBot.value_counts()

No          3027
Yes          690
Not sure      78
Name: isBot, dtype: int64

### 2.2.2. Separate X and y values
We use only 4 features here to create a baseline model. However, it is not enough to get good results.

In [123]:
X = dfBotAll_train[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites']]
y = dfBotAll_train.isBot.apply(lambda x: 1 if x=='Yes' else 0)

### 2.2.3. Train-test split

In [124]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

### 2.2.4. Train the model

In [125]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance
dtc_bot = DecisionTreeClassifier()

# fit your model
dtc_bot.fit(X_train, y_train)

# make predictions
preds = dtc_bot.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.35 
 Accuracy Score: 0.7552631578947369 
 Confusion Matrix: 
 [[541 101]
 [ 85  33]]


# 3. MAKE PREDICTIONS

Here, you will make predictions with the models that you have trained above.

## 3.1. Predictions for Tweets (Political or Not)

In [128]:
# read the evaluation file as follows
evaluationTweetDf = pd.read_csv('evaluation-round{}-tweet.csv'.format(ROUND), dtype={0: str}, header=None, names=['tweet_id'])
evaluationTweetDf = evaluationTweetDf.dropna()
evaluationTweetDf

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfPolitical_test = dfPolitical.merge(evaluationTweetDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfPolitical_test[['num_political_entities','total_interactions','num_hashtags']]

# make predictions based on these variables
predictions_political = dtc_political.predict(X)

### This part is important! We expect you to return your predictions in the following format:

In [138]:
modelPredTweet = dict([(x,float(y)) for x,y in zip([*dfPolitical_test.tweet_id], predictions_political)])
modelPredTweet

TypeError: 'dict' object is not callable

## 3.2. Predictions for Users (Bot or Not)

In [ ]:
evaluationUserDf = pd.read_csv('evaluation-round{}-user.csv'.format(ROUND), dtype={0: str}, header=None, names=['user_screen_name'])
evaluationUserDf = evaluationUserDf.dropna()

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfBot_test = dfBotAll.merge(evaluationUserDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfBot_test[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites']]

# make predictions based on these variables
predictions_bot = dtc_bot.predict(X)

In [ ]:
modelPredUser = dict([(x,float(y)) for x,y in zip([*dfBot_test.user_screen_name], predictions_bot)])
modelPredUser

{'nedenburdaysam': 0.0,
 'biologselim': 1.0,
 'alaraaynncnm': 0.0,
 '_sydneycarton_': 0.0,
 'denizlihabercom': 0.0,
 'burakerbaychp': 0.0,
 'mustafaarst': 0.0,
 'mvnez': 0.0,
 'qara118': 0.0,
 'alpar_kaan': 0.0,
 'farukhalit2': 0.0,
 'haf_zhan': 1.0,
 'harlunoshi': 0.0,
 'heritagepaix': 1.0,
 '37baho37': 0.0,
 'tamerduran_1': 0.0,
 'donkisotumsu': 0.0,
 'nuranwolf': 0.0,
 'politikgundem': 1.0,
 'isakethudax': 0.0,
 'ilaydejaneiro': 0.0,
 'gendenmukatol': 0.0,
 '1905anason': 1.0,
 'eraydurgut03': 1.0,
 'dasiskein': 0.0,
 'mett_1907': 0.0,
 'semihyeteer': 0.0,
 'haberinyokcokk': 0.0,
 'meleky_ozaydin': 0.0,
 'han34nesli': 0.0,
 'bilobi4': 0.0,
 'berkeduranovic': 0.0,
 'cagdasadim': 0.0,
 'merabalare': 0.0,
 'sevdaac72373936': 0.0,
 '21gramlife1': 0.0,
 'cakan0_': 0.0,
 'oguzksalici': 0.0,
 'emre_caliskann': 0.0,
 'mehmet07454846': 0.0,
 'lifegs': 1.0,
 'begumkarabeyx': 1.0,
 'avutulan': 0.0,
 'imamgibiimam': 1.0,
 'durdane52': 0.0,
 'radyotrafik35': 0.0,
 'rk_ozanyali': 1.0,
 'uykusuz75'

# PREPARE SUBMISSION

You will need to submit exact same file produced by using the following code. Any deviation from the desired format willbe marked as 0.

In [126]:
# Explain your approach

data_explanations = '''
Explain here how you handle data for training
'''

feature_explanations = '''
What types of features you created

raw data extraction:
check_if_reply
get_user_location
get_verified
get_age
get_friends_followers
get_tweet_count
get_listed_count

extracted features:
UsernameSpecialCharacterCount
FriendsVFollowers
TweetsPerDay
'''

model_explanations = '''
What did you try and used for modeling
'''

additional_explanations = '''
Any other tricks that you tried for the project
'''


In [127]:
predictions = {
    'round': ROUND,
    'student_id': STUDENT_ID,
    'user_predictions': modelPredUser,
    'tweet_predictions': modelPredTweet,
    'explanations': {
        'data': data_explanations,
        'feature': feature_explanations,
        'model': model_explanations,
        'other': additional_explanations,
    }
}


with open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'w') as fl:
    fl.write(json.dumps(predictions, indent=4))

NameError: name 'modelPredUser' is not defined

In [ ]:
# Test your submission file

submission = json.load(open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'r'))
submission

{'explanations': {'data': '\nExplain here how you handle data for training\n',
  'feature': '\nWhat types of features you created\n',
  'model': '\nWhat did you try and used for modeling\n',
  'other': '\nAny other tricks that you tried for the project\n'},
 'round': 1,
 'student_id': 'baseline',
 'tweet_predictions': {'1593649159009099777': 1.0,
  '1367571642604544000': 1.0,
  '1589993032975544320': 1.0,
  '1565312596135354373': 1.0,
  '1388235183653011462': 1.0,
  '1592120408073203712': 0.0,
  '1439547067337256967': 1.0,
  '1597274845381029888': 1.0,
  '1586021183958704128': 1.0,
  '1356926480605982728': 0.0,
  '1595357036925026306': 1.0,
  '1585766233491886081': 1.0,
  '1595871258985615361': 1.0,
  '1352635736537882629': 1.0,
  '1583477966373543936': 1.0,
  '1564926450096013313': 1.0,
  '1585634359612420101': 1.0,
  '1384499047390658560': 0.0,
  '1596583748669419521': 1.0,
  '1391681495622995971': 0.0,
  '1365710259549966339': 1.0,
  '1590673118397624323': 1.0,
  '138995194334331699